In [1]:
import torch
from torch.utils.data import DataLoader


# move necessary helper functions

In [2]:
from update_utilities import update_utilities_class

In [3]:
import os
update_utilities_class(file_name="Transformer.py",current_path=os.getcwd()).run()
update_utilities_class(file_name="custom_text_dataset.py",current_path=os.getcwd()).run()
update_utilities_class(file_name="loss_functions.py",current_path=os.getcwd()).run()
update_utilities_class(file_name="train_test_loop.py",current_path=os.getcwd()).run()

File already exist in destination folder, it is now removed
File copied, now the file is available to import from the destinated path
File already exist in destination folder, it is now removed
File copied, now the file is available to import from the destinated path
File already exist in destination folder, it is now removed
File copied, now the file is available to import from the destinated path
File already exist in destination folder, it is now removed
File copied, now the file is available to import from the destinated path


# prepare data pipeline

In [2]:
from general_functions import HelperFunctionsClass
h = HelperFunctionsClass()

In [3]:
train_data = h.convert_str_file_to_int_array(file_path="training_data/train_tokens_vocab2000.txt",convert_to_torch=True)

In [4]:
val_data = h.convert_str_file_to_int_array(file_path="training_data/val_tokens_vocab2000.txt",convert_to_torch=True)

In [5]:
len(train_data), len(val_data)

(5101623, 561744)

In [6]:
from custom_text_dataset import slideTokenizedTextDataset

In [7]:
block_size = 400
train_dataset = slideTokenizedTextDataset(full_txt=train_data, block_size=block_size)
val_dataset = slideTokenizedTextDataset(full_txt=val_data, block_size=block_size)

In [8]:
len(train_dataset), len(val_dataset)

(5101223, 561344)

In [9]:
batch_size = 64
train_num_samples = 600000
train_sampler = torch.utils.data.RandomSampler(train_dataset,replacement=False,num_samples=train_num_samples)
train_loader = DataLoader(train_dataset,batch_size=batch_size,sampler=train_sampler,drop_last=True)

In [10]:
val_num_samples = 100000
val_sampler = torch.utils.data.RandomSampler(val_dataset,replacement=False,num_samples=val_num_samples)
val_loader = DataLoader(val_dataset,sampler=val_sampler,batch_size=batch_size,drop_last=True)

In [11]:
len(train_loader), len(val_loader)

(9375, 1562)

# Model

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [10]:
import Transformer

In [11]:
vocab_size = 2003 # 2000 vocab size + 3 special tokens
transformer = Transformer.TransformerClass(vocab_size=vocab_size,emb_dim=512,num_heads=8,n_layer=12,block_size=400, ff_multiplier=4,
                                           dropout_rate_attention=0.1, dropout_rate_ff=0.2, dropout_rate_pos_enc=0).to(device)

In [12]:
num_para = sum(p.numel() for p in transformer.parameters()) / 1e6
print(f"Total number of Parameters {round(num_para,2)} million")

Total number of Parameters 39.86 million


In [13]:
transformer.load_state_dict(torch.load(r"baseline_vocab2000_12layer_400context stats\baseline_vocab2000_12layer_400context weights\baseline_vocab2000_12layer_400context_last_run4.pth"))

<All keys matched successfully>

# Training stage 1
**Rapid training with relatively higher lr, more validation check point, relatively lower number of validation data**

In [14]:
from train_test_loop import train_test_loop_class

In [19]:
lr = 1e-4
optimizer = torch.optim.AdamW(params=transformer.parameters(),lr=lr)

In [20]:
overwrite=True
num_epochs=1
print_every = 855

train_loop = train_test_loop_class(model=transformer,train_loader=train_loader,val_loader=val_loader,test_loader=None,
                                   epochs=num_epochs, print_every_n_batch=print_every,device=device,
                                   model_name="baseline_vocab2000_12layer_400context",optimizer = optimizer, calculate_accuracy=False,
                                   overwrite_message=overwrite,problem_type="Multiclass Classification",
                                   update_loss_fn=False, print_result=True, print_full=False)

In [21]:
train_loop.train()

  0%|          | 0/9375 [00:00<?, ?it/s]

Batch: 855 / 9375 || Average per-Batch Training Loss: 4.5761 || Average per-Batch Validation Loss: 4.0578
Batch: 1710 / 9375 || Average per-Batch Training Loss: 3.8726 || Average per-Batch Validation Loss: 3.6186
Batch: 2565 / 9375 || Average per-Batch Training Loss: 3.5018 || Average per-Batch Validation Loss: 3.3727
Batch: 3420 / 9375 || Average per-Batch Training Loss: 3.2738 || Average per-Batch Validation Loss: 3.2481
Batch: 4275 / 9375 || Average per-Batch Training Loss: 3.1178 || Average per-Batch Validation Loss: 3.1699
Batch: 5130 / 9375 || Average per-Batch Training Loss: 2.9958 || Average per-Batch Validation Loss: 3.1205
Batch: 5985 / 9375 || Average per-Batch Training Loss: 2.8905 || Average per-Batch Validation Loss: 3.1004
Batch: 6840 / 9375 || Average per-Batch Training Loss: 2.7986 || Average per-Batch Validation Loss: 3.0873
Batch: 7695 / 9375 || Average per-Batch Training Loss: 2.7124 || Average per-Batch Validation Loss: 3.0889
Batch: 8550 / 9375 || Average per-Batc

In [22]:
train_loop.overwrite_message = False

In [25]:
batch_size = 64
train_num_samples = 700000
train_sampler = torch.utils.data.RandomSampler(train_dataset,replacement=False,num_samples=train_num_samples)
train_loader = DataLoader(train_dataset,batch_size=batch_size,sampler=train_sampler,drop_last=True)
val_num_samples = 116000
val_sampler = torch.utils.data.RandomSampler(val_dataset,replacement=False,num_samples=val_num_samples)
val_loader = DataLoader(val_dataset,sampler=val_sampler,batch_size=batch_size,drop_last=True)
len(train_loader), len(val_loader)

(10937, 1812)

In [26]:
train_loop.train_loader = train_loader
train_loop.val_loader = val_loader
train_loop.print_progress = 1000

In [27]:
train_loop.train()

  0%|          | 0/10937 [00:00<?, ?it/s]

Batch: 1000 / 10937 || Average per-Batch Training Loss: 3.9907 || Average per-Batch Validation Loss: 3.9026
Batch: 2000 / 10937 || Average per-Batch Training Loss: 3.9360 || Average per-Batch Validation Loss: 3.8435
Batch: 3000 / 10937 || Average per-Batch Training Loss: 3.8818 || Average per-Batch Validation Loss: 3.7849
Batch: 4000 / 10937 || Average per-Batch Training Loss: 3.8277 || Average per-Batch Validation Loss: 3.7312
Batch: 5000 / 10937 || Average per-Batch Training Loss: 3.7759 || Average per-Batch Validation Loss: 3.6801
Batch: 6000 / 10937 || Average per-Batch Training Loss: 3.7260 || Average per-Batch Validation Loss: 3.6347
Batch: 7000 / 10937 || Average per-Batch Training Loss: 3.6819 || Average per-Batch Validation Loss: 3.5933
Batch: 8000 / 10937 || Average per-Batch Training Loss: 3.6401 || Average per-Batch Validation Loss: 3.5548
Batch: 9000 / 10937 || Average per-Batch Training Loss: 3.6003 || Average per-Batch Validation Loss: 3.5222
Batch: 10000 / 10937 || Aver

# Training Stage 2
**slow fine-tuning with relatively lower lr, less validation check point, relatively higher number of validation data**

In [15]:
lr = 2e-6
optimizer = torch.optim.AdamW(params=transformer.parameters(),lr=lr)

In [16]:
batch_size = 64
train_num_samples = 800000
train_sampler = torch.utils.data.RandomSampler(train_dataset,replacement=False,num_samples=train_num_samples)
train_loader = DataLoader(train_dataset,batch_size=batch_size,sampler=train_sampler,drop_last=True)
val_num_samples = 200000
val_sampler = torch.utils.data.RandomSampler(val_dataset,replacement=False,num_samples=val_num_samples)
val_loader = DataLoader(val_dataset,sampler=val_sampler,batch_size=batch_size,drop_last=True)
len(train_loader), len(val_loader)

(12500, 3125)

In [17]:
overwrite=False
num_epochs=1
print_every = 2500

train_loop = train_test_loop_class(model=transformer,train_loader=train_loader,val_loader=val_loader,test_loader=None,
                                   epochs=num_epochs, print_every_n_batch=print_every,device=device,
                                   model_name="baseline_vocab2000_12layer_400context",optimizer = optimizer, calculate_accuracy=False,
                                   overwrite_message=overwrite,problem_type="Multiclass Classification",
                                   update_loss_fn=False, print_result=True, print_full=False)

In [18]:
train_loop.train()

  0%|          | 0/12500 [00:00<?, ?it/s]

Batch: 2500 / 12500 || Average per-Batch Training Loss: 2.4663 || Average per-Batch Validation Loss: 3.0936
Batch: 5000 / 12500 || Average per-Batch Training Loss: 2.4609 || Average per-Batch Validation Loss: 3.0943
Batch: 7500 / 12500 || Average per-Batch Training Loss: 2.4551 || Average per-Batch Validation Loss: 3.0969
Batch: 10000 / 12500 || Average per-Batch Training Loss: 2.4499 || Average per-Batch Validation Loss: 3.0988
Batch: 12500 / 12500 || Average per-Batch Training Loss: 2.4443 || Average per-Batch Validation Loss: 3.1009

 All Done

Overall training took 1.86 hours



In [30]:
train_loop.overwrite_message = False

In [36]:
lr = 2e-6
optimizer = torch.optim.AdamW(params=transformer.parameters(),lr=lr)

In [38]:
train_loop.optimizer = optimizer

In [40]:
train_loop.model_name

'baseline_vocab2000_2xcontextlen'

In [18]:
#train_loop.model = transformer.to(device)
train_loop.train()

  0%|          | 0/12500 [00:00<?, ?it/s]

Batch: 2500 / 12500 || Average per-Batch Training Loss: 2.4386 || Average per-Batch Validation Loss: 3.1031
Batch: 5000 / 12500 || Average per-Batch Training Loss: 2.4347 || Average per-Batch Validation Loss: 3.1060
Batch: 7500 / 12500 || Average per-Batch Training Loss: 2.4284 || Average per-Batch Validation Loss: 3.1068
Batch: 10000 / 12500 || Average per-Batch Training Loss: 2.4233 || Average per-Batch Validation Loss: 3.1087
Batch: 12500 / 12500 || Average per-Batch Training Loss: 2.4186 || Average per-Batch Validation Loss: 3.1108

 All Done

Overall training took 1.85 hours



# Generator

In [19]:
import os
import torch
class generator:
    def __init__(self, model, encoder, decoder, model_name):
        self.model = model.cpu()
        self.encoder = encoder
        self.decoder = decoder
        self.model_name = model_name
        self.folder_name = "generated_text"
    
    def _open_file(self):
        if not os.path.exists(self.folder_name):
            os.makedirs(self.folder_name)
        self.f = open(os.path.join(self.folder_name,self.model_name)+".txt","a")

    
    def generate_without_prompt(self, user_input, generation_length=300, block_size=512,default_start_token=True,pad_with=32):
        self._open_file()
        output_list = [user_input]
        if default_start_token: user_input = '<|startofchapter|>' + user_input
        # tokenize user input
        input_tokens = torch.tensor(self.encoder(user_input),dtype=torch.long)
        # pad tokens
        if len(input_tokens) < block_size:
            tokens = torch.full(size=(1,block_size),fill_value=pad_with,dtype=torch.long)
            tokens[0,-len(input_tokens):] = input_tokens
        else:
            tokens = input_tokens[-block_size:].unsqueeze(0)
        
        m = f"User input: {user_input}\nGenerating----------------------------------------------------\n"
        print(m)
        self.f.write("\n\n"+m+"\n")

        print_status = False
        print_idx_start = 0
        for i in range(generation_length):
            if i % 30 == 1:
                print_status = True
            if ("." in output_list[-1] or "," in output_list[-1]) and print_status==True:
                output_sequence = "".join(output_list[print_idx_start:])
                print_idx_start = len(output_list)
                print_status=False
                print(output_sequence)
                self.f.write(output_sequence+"\n")
            tokens_truncate = tokens[0,-block_size:]
            logit = self.model(tokens_truncate)
            logit = logit[0,-1,:]
            prob = torch.nn.functional.softmax(logit,dim=0)
            new_token = torch.multinomial(input=prob,num_samples=1)
            tokens = torch.cat((tokens,new_token.unsqueeze(0)),dim=1)
            output_list.append(self.decoder([new_token.item()]))
        m = f"\nEnd of generation------------------------------------------------------------------\n"
        print(m)
        self.f.write(m+"\n\n")
        self.f.close()
        return output_list
            


        

In [20]:
from regex_bpe_tokenizer import ApplyTokenizer
import os
tokenizer = ApplyTokenizer(title="FantasyGPTv1",vocab_size=2000,tokenizer_folder_path=os.getcwd())
len(tokenizer.vocab)


2000

In [21]:
gen = generator(transformer,tokenizer.encode,tokenizer.decode,"baseline_vocab2000_12layer_400context_run5")

In [25]:
user_input="There was an incident at the school, "
output = gen.generate_without_prompt(user_input=user_input,generation_length=300,default_start_token=True,block_size=400)

User input: <|startofchapter|>There was an incident at the school, 
Generating----------------------------------------------------

There was an incident at the school, utter or illusions that awoke me spelled on,
 bringing up my things. I only knew the cabin in the corner of my eye when preened to spend my time here and no one but me.
 My eyes burned with fragile as my fingers reached for the bed,
 some socket perched on the studio level on the floor. She pulled a small cup for it.
 "Hidden marks from your tent," a growling nod. Anyone but the female kept screaming completely wanyverns poured out.
 "They brought me in a prison. And I gave them more of a sadist."
 It wasn't his failure that had long since vanished from utter silence, that sight of her.
 Bile thing that smell ... on mighty, she couldn't read it as if it were possible to be clothed registering.
 The broken dish proximity of thread within her demeanor. No one could steal it, used all sense of hers.
 "Actually, I think," a